In [1]:
!pip install ipykernel transformer_lens plotly 

In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=7

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=7


In [1]:
import transformer_lens
import torch
# import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix
# Import stuff
import torch
import torch.nn as nn
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import plotly.express as px
import numpy as np

from jaxtyping import Float
from functools import partial
import numpy as np

/nlp/projekty/aver/xspiege1/algorithmic_reasoning/.env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Convert HF Transformers weights to HookedTransformer weights

In [2]:
"""
from transformer_lens.pretrained.weight_conversions.llama import convert_llama_weights
from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_config(config)
model.load_state_dict(torch.load("checkpoints/llama_instruct_flip_flop_finetuned_grok.weights", weights_only=True))
hooked_config = transformer_lens.loading.get_pretrained_model_config("meta-llama/Llama-3.2-1B-Instruct")
new_state_dict = convert_llama_weights(model, hooked_config)
torch.save(new_state_dict, "checkpoints/llama_instruct_flip_flop_finetuned_grok_hooked.weights")
"""


'\nfrom transformer_lens.pretrained.weight_conversions.llama import convert_llama_weights\nfrom transformers import AutoConfig, AutoModelForCausalLM\n\nconfig = AutoConfig.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")\nmodel = AutoModelForCausalLM.from_config(config)\nmodel.load_state_dict(torch.load("checkpoints/llama_instruct_flip_flop_finetuned_grok.weights", weights_only=True))\nhooked_config = transformer_lens.loading.get_pretrained_model_config("meta-llama/Llama-3.2-1B-Instruct")\nnew_state_dict = convert_llama_weights(model, hooked_config)\ntorch.save(new_state_dict, "checkpoints/llama_instruct_flip_flop_finetuned_grok_hooked.weights")\n'

In [3]:
config = transformer_lens.loading.get_pretrained_model_config("meta-llama/Llama-3.2-1B-Instruct")
model = transformer_lens.HookedTransformer(config)
model.load_and_process_state_dict(torch.load("checkpoints/llama_instruct_flip_flop_finetuned_grok_hooked.weights", weights_only=True))
#config = transformer_lens.loading.get_pretrained_model_config("Qwen/Qwen2.5-1.5B-Instruct")
#model = transformer_lens.HookedTransformer(config)
#model.load_and_process_state_dict(torch.load("checkpoints/Qwen2.5-1.5B-Instruct_value_assign_finetuned_hooked.weights", weights_only=True))


In [4]:
charset_from = ""
for id in range(0, 50000):
    token = model.tokenizer.decode(id, skip_special_tokens=True)
    if len(token) > 1 or token == "�":
        continue
    decode_id = model.tokenizer.encode(token, add_special_tokens=False)
    if len(decode_id) == 1 and decode_id[0] == id:
        charset_from += token

In [5]:
charset_from[:105]

'!"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n'

In [6]:
import sys
sys.path.insert(0, "..")
sys.path.insert(0, "../generators")
sys.path.insert(0, "generators")


from generators.flip_flop_languages import FlipFlopGenerator
import transformers
import itertools


data_generator = FlipFlopGenerator(tokenizer=model.tokenizer,
                                       seed=0,
                                       use_few_shot=True, 
                                       use_instruction=True, 
                                       apply_chat_template=True,
                                       length=(500,500),
                                       )
samples = itertools.islice(data_generator.generate_samples(), 100)

In [7]:
samples = list(samples)
texts = [sample[0] for sample in samples]
input_ids = [sample[1] for sample in samples]
target_ids = [sample[2] for sample in samples]
attn_labels = [sample[3] for sample in samples]

In [8]:
texts[0]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 16 May 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n\nYou are an expert in solving tasks requiring manipulation of symbols. Your task will be to evaluate a string representing a register.\nThe string is composed of commands to the register: write, read, ignore, flip. Each is represented by a single character (w, r, i, f)\nDo not generate any words. You can only generate the symbols of the reversed string of symbols and nothing more.\n\nHere are some examples:\nw11i11f10r10f10r11i11i11i11i11i11i11r10\nUse them to solve the following task:\nw21i61i20f11w60f61f50i00f21f01i20f31f20f00f61f60f31w51f60w40w60f30w21w21f01f11i01i40i10w01i00f60w00f61f41f60w51f21i51w20i41w10f20f11w21w00f10f50w00f60i01w00w10i10f01f01w10f21i10f30f01w21f30f50w60w21w41f10i51f61f21f20f50i50i50f20w61i41f20f21f30w40f20w11i51w31f50w30i50i41f40w31i61w61w40f10i51i00w50f40w00i20w31f00i30i11w60i6

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import random

def visualize_attention_heads(attention_matrix, tokens=None, title="Attention Heatmap", save_path=None, heads=None, length=None, attn_labels=None):
    """
    Visualize a multi-head attention matrix as heatmaps for each head.

    Parameters:
        attention_matrix (torch.Tensor): Tensor of shape (1, num_heads, seq_len, seq_len).
        tokens (list, optional): List of tokens corresponding to the rows/columns of the matrix.
        title (str): Title of the heatmap.
        save_path (str, optional): Path to save the heatmap images. If None, the heatmaps are not saved.
        heads (list, optional): List of head indices to visualize.
        length (int, optional): Number of tokens to display from the lower-right corner.
    """
    # Detach and convert to NumPy
    scores = attention_matrix.squeeze(0).cpu().detach().numpy()  # Shape: (num_heads, seq_len, seq_len)
    num_heads = scores.shape[0]
    seq_len = scores.shape[1]

    for head in range(num_heads):
        if heads is not None and head not in heads:
            continue
        head_scores = scores[head]  # Shape: (seq_len, seq_len)

        # Apply filtering for the lower-right "length x length" square
        if length is not None and length < seq_len:
            head_scores = head_scores[-length:, -length:]
            attn_labels = attn_labels[-length:, -length:]

        min_score = head_scores.min()
        max_score = head_scores.max()
        normalized = (head_scores - min_score) / (max_score - min_score)  # Normalize to [0, 1]

        fig, ax = plt.subplots(figsize=(10, 8))
        plt.imshow(normalized, cmap="viridis", interpolation="nearest")
        plt.colorbar(label="Attention Score")
        plt.title(f"{title} - Head {head + 1}")

        linewidth=1
        # Draw borders for attention label regions
        for i in range(length):
            for j in range(length):
                if attn_labels[i, j] == 1:
                    # Left border
                    if j == 0 or attn_labels[i, j-1] != 1:
                        ax.plot([j - 0.5, j - 0.5], [i - 0.5, i + 0.5], color='red', linewidth=linewidth, alpha=1)
                    # Right border
                    if j == length - 1 or attn_labels[i, j+1] != 1:
                        ax.plot([j + 0.5, j + 0.5], [i - 0.5, i + 0.5], color='red', linewidth=linewidth, alpha=1)
                    # Top border
                    if i == 0 or attn_labels[i-1, j] != 1:
                        ax.plot([j - 0.5, j + 0.5], [i - 0.5, i - 0.5], color='red', linewidth=linewidth, alpha=1)
                    # Bottom border
                    if i == length - 1 or attn_labels[i+1, j] != 1:
                        ax.plot([j - 0.5, j + 0.5], [i + 0.5, i + 0.5], color='red', linewidth=linewidth, alpha=1)


        plt.xlabel("Tokens")
        plt.ylabel("Tokens")
        plt.tight_layout()

        if save_path:
            plt.savefig(f"{save_path}_head_{head + 1}.png")
        plt.show()

In [10]:
best_heads = {}
def compute_scores(logits, target):
    ignore_index = -100  # Optional: index to ignore in loss/accuracy computation
    # Compute loss
    loss = torch.nn.functional.cross_entropy(
    logits.view(-1, logits.size(-1)),  # Flatten logits to (batch_size * seq_len, vocab_size)
    target.view(-1),               # Flatten target_ids to (batch_size * seq_len)
    ignore_index=ignore_index          # Ignore padding tokens if applicable
)

    # Compute accuracy
    predictions = torch.argmax(logits, dim=-1)  # Get predicted token indices
    correct = (predictions == target) & (target != ignore_index)  # Ignore padding tokens
    accuracy = correct.sum().item() / (target != ignore_index).sum().item()

    print(f"Loss: {loss.item()}, Accuracy: {accuracy:.2%}")

    errors = (predictions != target) & (target != ignore_index)  # Identify error positions
    errors_np = errors.cpu().numpy()  # Convert to NumPy for visualization
    #print("Errors:", errors_np)

    return loss.item(), accuracy

def fix_attn_pattern_hook(
    value: Float[torch.Tensor, "batch pos head_index d_head"],
    hook: HookPoint,
    attn_labels_1_idx,
    heads_to_reinforce,
    attn_labels,
    keep_last_n_tokens,
) -> Float[torch.Tensor, "batch pos head_index d_head"]:
    batch, heads, seq_len, _ = value.shape 
    layer = int(hook.name.split(".")[1])
    #if layer not in heads_to_reinforce.keys():
    #    return
    for h in range(heads):
        #if h not in heads_to_reinforce[layer]:
        #    continue
        acc = 0
        
        for i in range(seq_len):
            for j in range(seq_len):
                if (i, j) in attn_labels_1_idx:
                    value[0, h, i, j] += 0.1
                    #acc += value[0, h, i, j]
                    pass
        
        #visualize_attention_heads(value, None, title=f"Example Attention Heatmap Layer {layer}", heads=[h], length=keep_last_n_tokens, attn_labels=attn_labels)

    return value

device = "cuda"

is_attn_scores = lambda name: name.endswith("hook_pattern")
ablated = []
for idx in range(0, 10):



    
    indices =  np.argwhere(attn_labels[idx].cpu().numpy() == 1)
    attn_labels_1_idx = set(map(tuple, indices))
    

    model.to(device)
    input_ids[idx] = input_ids[idx].to(device)
    target_ids[idx] = target_ids[idx].to(device)
    attn_labels[idx] = attn_labels[idx].to(device)
                
    heads_to_reinforce_x = [
        (9, [5, 7, 20, 25, 29, 30, 31]),
        (10, [8]),
        (11, [9, 8, 30]),
        (13, [9, 11]),
        (14, [24, 26, 27]),
        (15, [1, 8, 10]),

    ]

    cols_to_keep = ~np.all(attn_labels[idx].cpu().numpy() == -100, axis=0)
    keep_last_n_tokens = 70
    heads_x = dict(heads_to_reinforce_x)
    print(heads_x)
    logits = model.run_with_hooks(input_ids[idx], return_type="logits",
                                    fwd_hooks=[
                                        (is_attn_scores, partial(fix_attn_pattern_hook, attn_labels_1_idx=attn_labels_1_idx, heads_to_reinforce=heads_x, attn_labels=attn_labels[idx], keep_last_n_tokens = keep_last_n_tokens))
                                    ])
    loss, acc = compute_scores(logits, target_ids[idx])
    ablated.append((heads_x, loss, acc))
    del logits
    torch.cuda.empty_cache()
    input_ids[idx] = input_ids[idx].to("cpu")
    target_ids[idx] = target_ids[idx].to("cpu")
    attn_labels[idx] = attn_labels[idx].to("cpu")
    

Moving model to device:  cuda
{9: [5, 7, 20, 25, 29, 30, 31], 10: [8], 11: [9, 8, 30], 13: [9, 11], 14: [24, 26, 27], 15: [1, 8, 10]}
Loss: 0.45744937658309937, Accuracy: 50.00%
Moving model to device:  cuda
{9: [5, 7, 20, 25, 29, 30, 31], 10: [8], 11: [9, 8, 30], 13: [9, 11], 14: [24, 26, 27], 15: [1, 8, 10]}
Loss: 0.8426240682601929, Accuracy: 50.00%
Moving model to device:  cuda
{9: [5, 7, 20, 25, 29, 30, 31], 10: [8], 11: [9, 8, 30], 13: [9, 11], 14: [24, 26, 27], 15: [1, 8, 10]}
Loss: 0.006217836402356625, Accuracy: 100.00%
Moving model to device:  cuda
{9: [5, 7, 20, 25, 29, 30, 31], 10: [8], 11: [9, 8, 30], 13: [9, 11], 14: [24, 26, 27], 15: [1, 8, 10]}


KeyboardInterrupt: 

In [11]:
sums = 0
for item in ablated:
    sums += item[2]
print(sums/len(ablated))
print(sums)

0.85
25.5


In [ ]:
sorted(best_heads.items(), key=lambda item: item[1], reverse=True)

[(('blocks.11.attn.hook_pattern', 9),
  tensor(29.8965, device='cuda:0', grad_fn=<AddBackward0>)),
 (('blocks.14.attn.hook_pattern', 24),
  tensor(29.8524, device='cuda:0', grad_fn=<AddBackward0>)),
 (('blocks.11.attn.hook_pattern', 8),
  tensor(29.3969, device='cuda:0', grad_fn=<AddBackward0>)),
 (('blocks.9.attn.hook_pattern', 30),
  tensor(29.0661, device='cuda:0', grad_fn=<AddBackward0>)),
 (('blocks.13.attn.hook_pattern', 9),
  tensor(29.0473, device='cuda:0', grad_fn=<AddBackward0>)),
 (('blocks.9.attn.hook_pattern', 29),
  tensor(28.1519, device='cuda:0', grad_fn=<AddBackward0>)),
 (('blocks.15.attn.hook_pattern', 1),
  tensor(26.8898, device='cuda:0', grad_fn=<AddBackward0>)),
 (('blocks.9.attn.hook_pattern', 25),
  tensor(26.2973, device='cuda:0', grad_fn=<AddBackward0>)),
 (('blocks.15.attn.hook_pattern', 8),
  tensor(26.1444, device='cuda:0', grad_fn=<AddBackward0>)),
 (('blocks.9.attn.hook_pattern', 5),
  tensor(24.7707, device='cuda:0', grad_fn=<AddBackward0>)),
 (('blocks